In [9]:
import cv2
import time
import torch
import numpy as np
import os
import math
from collections import deque
from utils.datasets import letterbox
from utils.torch_utils import select_device
from models.experimental import attempt_load
from utils.plots import output_to_keypoint, plot_skeleton_kpts
from utils.general import non_max_suppression_kpt, strip_optimizer
from torchvision import transforms

class FallDetector:
    def __init__(self, poseweights='yolov7-w6-pose.pt', device='0'):
        """
        Initialize the Fall Detector with parameters as defined in the paper
        "Enhanced Fall Detection Using YOLOv7-W6-Pose for Real-Time Elderly Monitoring"
        
        Key parameters:
        - LENGTH_FACTOR_ALPHA (α): Used in height condition formula (Section 3.1)
        - VELOCITY_THRESHOLD: Threshold for fall speed detection (Section 3.2)
        - LEG_ANGLE_THRESHOLD: Degrees threshold for leg angles (Section 3.2)
        - TORSO_ANGLE_THRESHOLD: Degrees threshold for torso orientation (Section 3.2)
        - ASPECT_RATIO_THRESHOLD: Width/height ratio threshold (Section 3.1)
        - CONFIDENCE_THRESHOLD: Minimum keypoint confidence for reliable detection
        """
        print(f"Initializing Fall Detector with weights: {poseweights} on device: {device}")
        
        # Select the appropriate device
        self.device = select_device(device)
        self.half = self.device.type != 'cpu'
        
        # Load model
        self.model = attempt_load(poseweights, map_location=self.device)
        self.model.eval()
        
        # Create output directory if it doesn't exist
        os.makedirs('output', exist_ok=True)
        
        # Threshold parameters as defined in the paper
        self.LENGTH_FACTOR_ALPHA = 0.5  # α in the height condition formula
        self.VELOCITY_THRESHOLD = 1.0    # px/frame for fall speed
        self.LEG_ANGLE_THRESHOLD = 45    # degrees for leg angles
        self.TORSO_ANGLE_THRESHOLD = 50  # degrees for torso orientation
        self.ASPECT_RATIO_THRESHOLD = 0.8 # width/height ratio
        self.CONFIDENCE_THRESHOLD = 0.4  # minimum keypoint confidence
        self.TARGET_FPS = 25
        
        # State tracking variables
        self.prev_keypoints = None
        self.velocity_buffer = deque(maxlen=3)  # tracks vertical speed
        self.fall_buffer = deque(maxlen=2)      # confirmation buffer
        self.prev_frame_time = None
        self.fall_start_time = None
        self.prev_shoulder_y = None
        
        # Fall detection status
        self.fall_detected = False
    
    def calculate_euclidean_distance(self, point1, point2):
        """
        Calculate Euclidean distance between two points
        Used in the paper to measure distances between key body points,
        particularly for the Lfactor (length factor) calculation in Section 3.1
        
        Args:
            point1, point2: Coordinate points (x,y)
        Returns:
            Euclidean distance between the points
        """
        return math.hypot(point1[0]-point2[0], point1[1]-point2[1])

    def calculate_angle(self, a, b, c):
        """
        Calculate angle between three points (in degrees)
        Used in the paper for calculating leg angles (Section 3.2)
        
        Args:
            a, b, c: Three points where b is the vertex
        Returns:
            Angle in degrees
        """
        try:
            ba = np.array([a[0]-b[0], a[1]-b[1]])
            bc = np.array([c[0]-b[0], c[1]-b[1]])
            cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc) + 1e-6)
            return np.degrees(np.arccos(np.clip(cosine_angle, -1.0, 1.0)))
        except:
            return 180  # return maximum angle if calculation fails

    def calculate_torso_angle(self, shoulders, hips):
        """
        Calculate torso angle relative to vertical axis
        Implements the torso orientation assessment described in Section 3.2
        of the paper to detect when the torso is horizontal (fallen state)
        
        Args:
            shoulders: list of shoulder points [(x,y), (x,y)]
            hips: list of hip points [(x,y), (x,y)]
        Returns:
            angle in degrees between torso and vertical axis
        """
        shoulder_center = np.mean(shoulders, axis=0)
        hip_center = np.mean(hips, axis=0)
        vertical_vector = np.array([0, 1])
        torso_vector = np.array([hip_center[0]-shoulder_center[0], 
                                hip_center[1]-shoulder_center[1]])
        
        if np.linalg.norm(torso_vector) < 1e-6:
            return 90  # neutral angle if points overlap
            
        cosine = np.dot(torso_vector, vertical_vector) / (np.linalg.norm(torso_vector) + 1e-6)
        return np.degrees(np.arccos(np.clip(cosine, -1.0, 1.0)))

    def detect_fall(self, keypoints):
        """
        Main fall detection function implementing the paper's algorithm from Sections 3.1 and 3.2
        Combines multiple conditions (height, velocity, angles, aspect ratio) to detect falls
        
        Args:
            keypoints: Array of 17 keypoints with (x,y,confidence)
        Returns:
            tuple: (is_fall, state, condition_info)
        """
        # Keypoint indices as defined in the paper
        NOSE = 0
        LEFT_SHOULDER = 5
        RIGHT_SHOULDER = 6
        LEFT_HIP = 11
        RIGHT_HIP = 12
        LEFT_KNEE = 13
        RIGHT_KNEE = 14
        LEFT_ANKLE = 15
        RIGHT_ANKLE = 16
        
        try:
            # Extract keypoints with confidence check
            kp = {}
            
            # Reshape keypoints to get (x, y, conf) format for each keypoint
            reshaped_kpts = keypoints.reshape(-1, 3)
            
            # Extract specific keypoints
            kp['nose'] = reshaped_kpts[NOSE]
            kp['left_shoulder'] = reshaped_kpts[LEFT_SHOULDER]
            kp['right_shoulder'] = reshaped_kpts[RIGHT_SHOULDER]
            kp['left_hip'] = reshaped_kpts[LEFT_HIP]
            kp['right_hip'] = reshaped_kpts[RIGHT_HIP]
            kp['left_knee'] = reshaped_kpts[LEFT_KNEE]
            kp['right_knee'] = reshaped_kpts[RIGHT_KNEE]
            kp['left_ankle'] = reshaped_kpts[LEFT_ANKLE]
            kp['right_ankle'] = reshaped_kpts[RIGHT_ANKLE]
            
            # Confidence check for all keypoints
            if any(point[2] < self.CONFIDENCE_THRESHOLD for point in kp.values()):
                return False, "low_confidence", []

            # Get coordinates (convert to tuples for clarity)
            ls = (kp['left_shoulder'][0], kp['left_shoulder'][1])
            rs = (kp['right_shoulder'][0], kp['right_shoulder'][1])
            lh = (kp['left_hip'][0], kp['left_hip'][1])
            rh = (kp['right_hip'][0], kp['right_hip'][1])
            lk = (kp['left_knee'][0], kp['left_knee'][1])
            rk = (kp['right_knee'][0], kp['right_knee'][1])
            la = (kp['left_ankle'][0], kp['left_ankle'][1])
            ra = (kp['right_ankle'][0], kp['right_ankle'][1])

            """ 1. HEIGHT CONDITION (Paper Section 3.1) """
            # Calculate length factor (Lfactor) as Euclidean distance
            torso_mid = ((lh[0] + rh[0])/2, (lh[1] + rh[1])/2)
            Lfactor = self.calculate_euclidean_distance(ls, torso_mid)
            
            # Get vertical positions
            max_feet_y = max(la[1], ra[1])
            min_shoulder_y = min(ls[1], rs[1])
            
            # Paper's height condition: yl ≤ yFl + α·Lfactor
            height_cond = min_shoulder_y >= (max_feet_y - self.LENGTH_FACTOR_ALPHA * Lfactor)
            
            """ 2. VELOCITY CONDITION (Paper Section 3.2) """
            current_time = time.time()
            vertical_speed = 0
            current_min_y = min(ls[1], rs[1])
            
            if self.prev_shoulder_y is not None and self.prev_frame_time is not None:
                time_elapsed = current_time - self.prev_frame_time
                if time_elapsed > 0:
                    vertical_speed = (current_min_y - self.prev_shoulder_y) / time_elapsed
                    self.velocity_buffer.append(abs(vertical_speed))
            
            avg_speed = sum(self.velocity_buffer)/len(self.velocity_buffer) if self.velocity_buffer else 0
            speed_cond = avg_speed >= self.VELOCITY_THRESHOLD
            
            """ 3. ANGLE CONDITIONS (Paper Section 3.2) """
            left_leg_angle = self.calculate_angle(lh, lk, la)
            right_leg_angle = self.calculate_angle(rh, rk, ra)
            leg_angle_cond = min(left_leg_angle, right_leg_angle) < self.LEG_ANGLE_THRESHOLD
            
            # Torso orientation (not explicitly in paper but mentioned in text)
            torso_angle = self.calculate_torso_angle([ls, rs], [lh, rh])
            torso_cond = torso_angle > self.TORSO_ANGLE_THRESHOLD
            
            """ 4. ASPECT RATIO CONDITION (Paper Section 3.1) """
            # Body orientation ratio: width/height
            body_width = abs(ls[0] - rs[0])
            head_to_feet = abs(kp['nose'][1] - max_feet_y)
            orientation_ratio = body_width / (head_to_feet + 1e-6)
            aspect_cond = orientation_ratio > self.ASPECT_RATIO_THRESHOLD
            
            """ FALL DECISION LOGIC (Paper Section 3) """
            # Combined conditions - at least 2 must be true
            conditions_met = sum([height_cond, speed_cond, leg_angle_cond, torso_cond, aspect_cond])
            
            # State determination
            current_state = "normal"
            conditions_info = []
            
            if height_cond:
                if speed_cond:  # Rapid descent
                    current_state = "falling"
                    self.fall_start_time = current_time
                    conditions_info.append(f"speed:{avg_speed:.1f}px/s")
                elif torso_cond and self.fall_start_time and (current_time - self.fall_start_time < 1.0):
                    current_state = "fallen"
                    conditions_info.append("horizontal")
            
            if leg_angle_cond:
                conditions_info.append(f"leg_angle:{min(left_leg_angle, right_leg_angle):.0f}°")
            
            if aspect_cond:
                conditions_info.append(f"aspect:{orientation_ratio:.2f}")
            
            # Final decision with confirmation buffer
            is_fall = conditions_met >= 2
            self.fall_buffer.append(is_fall)
            final_detection = sum(self.fall_buffer) >= 2 if len(self.fall_buffer) >= 1 else is_fall
            
            if final_detection:
                current_state = "fallen"
                self.fall_detected = True
            else:
                self.fall_detected = False
            
            # Update tracking variables
            self.prev_keypoints = kp
            self.prev_shoulder_y = current_min_y
            self.prev_frame_time = current_time
            
            # Diagnostic information
            conditions_info.extend([
                f"height:{'Y' if height_cond else 'N'}",
                f"speed:{'Y' if speed_cond else 'N'}",
                f"leg_angle:{'Y' if leg_angle_cond else 'N'}",
                f"torso:{'Y' if torso_cond else 'N'}",
                f"aspect:{'Y' if aspect_cond else 'N'}",
                f"conf:{min(p[2] for p in kp.values()):.2f}"
            ])
            
            return final_detection, current_state, conditions_info
            
        except Exception as e:
            print(f"Detection error: {str(e)}")
            return False, "error", [f"Error: {str(e)}"]

    def process_frame(self, frame):
        """
        Process a single frame for fall detection
        
        Args:
            frame: Video frame to process
            
        Returns:
            frame: Processed frame with detections
            is_fall: Boolean indicating whether a fall was detected
            state: Current state (normal, falling, fallen)
            condition_info: List of conditions that triggered the detection
        """
        # Preprocess image
        orig_image = frame.copy()
        image = cv2.cvtColor(orig_image, cv2.COLOR_BGR2RGB)
        
        # Resize image while maintaining aspect ratio
        frame_height, frame_width = orig_image.shape[:2]
        image = letterbox(image, (frame_width), stride=64, auto=True)[0]
        
        # Convert to tensor
        image_ = image.copy()
        image = transforms.ToTensor()(image)
        image = torch.tensor(np.array([image.numpy()]))
        
        image = image.to(self.device)
        image = image.float()
        
        # Inference
        with torch.no_grad():
            output, _ = self.model(image)
            
        # Post-process
        output = non_max_suppression_kpt(output, 0.25, 0.65, nc=self.model.yaml['nc'], nkpt=self.model.yaml['nkpt'], kpt_label=True)
        output = output_to_keypoint(output)
        
        # Convert back to BGR for display
        img = image[0].permute(1, 2, 0) * 255
        img = img.cpu().numpy().astype(np.uint8)
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        
        # Initialize fall status and state for this frame
        is_fall = False
        current_state = "normal"
        condition_info = []
        
        # Process each person detected
        for idx in range(output.shape[0]):
            # Draw skeleton and keypoints
            plot_skeleton_kpts(img, output[idx, 7:].T, 3)
            
            # Calculate improved bounding box based on keypoints (YouTube approach)
            # Find the minimum and maximum x,y coordinates from all keypoints
            kpts = output[idx, 7:].reshape(-1, 3)
            
            # Initialize with first keypoint
            x_values = [kpt[0] for kpt in kpts if kpt[2] > 0.5]  # Only use keypoints with confidence > 0.5
            y_values = [kpt[1] for kpt in kpts if kpt[2] > 0.5]
            
            if x_values and y_values:  # Check if we have valid keypoints
                xmin, ymin = min(x_values), min(y_values)
                xmax, ymax = max(x_values), max(y_values)
                
                # Add padding to make bounding box a bit larger
                padding = 10
                xmin = max(0, xmin - padding)
                ymin = max(0, ymin - padding)
                xmax = xmax + padding
                ymax = ymax + padding
                
                # Calculate aspect ratio for reference (not used in detection)
                width = xmax - xmin
                height = ymax - ymin
                bbox_aspect_ratio = width / height if height > 0 else 0
                
                # Calculate center
                cx = int((xmin + xmax) // 2)
                cy = int((ymin + ymax) // 2)
                
                # For debugging: show aspect ratio on frame
                cv2.putText(img, f"Ratio: {bbox_aspect_ratio:.2f}", (10, 30), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
            else:
                # Fallback to original bounding box if no valid keypoints
                x1, y1, x2, y2 = output[idx, 0], output[idx, 1], output[idx, 2], output[idx, 3]
                xmin, ymin = x1, y1
                xmax, ymax = x2, y2
                cx, cy = int((x1 + x2) // 2), int((y1 + y2) // 2)
            
            # Get key points for this person
            key_points = output[idx, 7:]
            
            # Detect fall for this person using enhanced algorithm
            person_fall, person_state, person_conditions = self.detect_fall(key_points)
            
            # If any person is falling, set global fall status
            if person_fall:
                is_fall = True
                current_state = person_state
                condition_info = person_conditions
                
                # Add visual indication of fall
                status_text = f"FALL DETECTED: {person_state.upper()}"
                cv2.putText(img, status_text, (50, 50), 
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                
                # Draw the bounding box in red for a fall
                cv2.rectangle(img, (int(xmin), int(ymin)), (int(xmax), int(ymax)), (0, 0, 255), 2)
                
                # For YouTube-style visual, add a colored rectangle at the center
                cv2.rectangle(img, (cx-10, cy-10), (cx+10, cy+10), (84, 61, 247), -1)
                
                # Add condition info to the frame
                for i, cond in enumerate(person_conditions[:3]):  # Show first 3 conditions only
                    cv2.putText(img, cond, (10, 60 + i*25), 
                              cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 1)
            else:
                # Draw normal bounding box in green for no fall
                cv2.rectangle(img, (int(xmin), int(ymin)), (int(xmax), int(ymax)), (0, 255, 0), 1)
                
                # Show normal state
                cv2.putText(img, f"State: {person_state}", (10, 60), 
                          cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)
        
        return img, is_fall, current_state, condition_info

def run_fall_detection(poseweights='yolov7-w6-pose.pt', source='pose.mp4', device='cpu', display=True, save_output=True):
    """
    Run fall detection on a video or webcam feed
    
    Args:
        poseweights: Path to the YOLOv7 pose weights
        source: Path to video file or webcam ID (0, 1, etc.)
        device: Device to run inference on ('cpu' or '0', '1', etc. for GPU)
        display: Whether to show video with detections in real-time
        save_output: Whether to save the output video
    """
    # Initialize the fall detector
    detector = FallDetector(poseweights=poseweights, device=device)
    
    # Parse the input source
    input_path = source
    if source.isnumeric():
        input_path = int(source)
    
    # Open video capture
    cap = cv2.VideoCapture(input_path)
    if not cap.isOpened():
        print(f"Error: Could not open video source {source}")
        return
    
    # Get video properties
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    # Setup output video writer if requested
    out = None
    if save_output:
        if isinstance(input_path, int):
            # For webcam
            output_path = os.path.join('output', f"webcam_fall_detection.mp4")
        else:
            # For video file
            filename = os.path.basename(input_path).split('.')[0]
            output_path = os.path.join('output', f"{filename}_fall_detection.mp4")
        
        # Create VideoWriter
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))
        print(f"Output will be saved to: {output_path}")
    
    # Process video frames
    frame_count = 0
    total_fps = 0
    
    print("Starting fall detection...")
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        frame_count += 1
        print(f"Processing frame {frame_count}")
        
        # Process frame for fall detection
        start_time = time.time()
        processed_frame, is_fall, current_state, condition_info = detector.process_frame(frame)
        end_time = time.time()
        
        # Calculate FPS
        processing_fps = 1 / (end_time - start_time)
        total_fps += processing_fps
        
        # Resize processed frame to match original dimensions for display and saving
        processed_frame_resized = cv2.resize(processed_frame, (frame_width, frame_height))
        
        # Add FPS info
        cv2.putText(processed_frame_resized, f"FPS: {processing_fps:.2f}", (frame_width - 150, 30), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        
        # Display the frame if requested
        if display:
            cv2.imshow('Fall Detection', processed_frame_resized)
            
            # Exit on 'q' press
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        
        # Save frame to output video if requested
        if save_output and out is not None:
            out.write(processed_frame_resized)
    
    # Release resources
    cap.release()
    if save_output and out is not None:
        out.release()
    cv2.destroyAllWindows()
    
    # Print statistics
    if frame_count > 0:
        avg_fps = total_fps / frame_count
        print(f"Processed {frame_count} frames")
        print(f"Average FPS: {avg_fps:.2f}")
        if save_output:
            print(f"Output saved to: {output_path}")

def run_interactive():
    """
    Interactive function to run fall detection with user input
    """
    # Get the weights file
    poseweights = input("Enter path to weights file [default: yolov7-w6-pose.pt]: ") or "yolov7-w6-pose.pt"
    
    # Get device type
    use_gpu = input("Use GPU? (y/n) [default: y]: ").lower() or "y"
    if use_gpu == "y":
        device = input("Enter GPU device ID [default: 0]: ") or "0"
    else:
        device = "cpu"
    
    # Get source type
    print("\nSelect input source:")
    print("1: Video file")
    print("2: Webcam")
    source_choice = input("Enter choice [1/2]: ")
    
    if source_choice == "1":
        # Video file
        default_video = "sample_video.mp4"
        source = input(f"Enter video file path [default: {default_video}]: ") or default_video
        # Ask if user wants to display the processed video in real-time
        display_video = input("Display video with pose estimation in real-time? (y/n) [default: y]: ").lower() or "y"
    else:
        # Webcam
        cam_id = input("Enter webcam ID [default: 0]: ") or "0"
        source = cam_id
        display_video = "y"  # Always display for webcam
    
    # Ask if user wants to save the output video
    save_video = input("Save output video? (y/n) [default: y]: ").lower() or "y"
        
    print(f"\nRunning fall detection with:")
    print(f"- Weights: {poseweights}")
    print(f"- Device: {device}")
    print(f"- Source: {source}")
    print(f"- Display: {'Yes' if display_video == 'y' else 'No'}")
    print(f"- Save output: {'Yes' if save_video == 'y' else 'No'}")
    confirmation = input("\nConfirm? (y/n) [default: y]: ").lower() or "y"
    
    if confirmation == "y":
        # Run the model
        run_with_display = (display_video == "y")
        save_output = (save_video == "y")
        
        # First strip optimizer to ensure model works correctly
        strip_optimizer(device, poseweights)
        
        # Run fall detection
        run_fall_detection(
            poseweights=poseweights,
            source=source,
            device=device,
            display=run_with_display,
            save_output=save_output
        )
    else:
        print("Operation cancelled")

if __name__ == "__main__":
    # Run interactively
    run_interactive()

Enter path to weights file [default: yolov7-w6-pose.pt]:  
Use GPU? (y/n) [default: y]:  
Enter GPU device ID [default: 0]:  



Select input source:
1: Video file
2: Webcam


Enter choice [1/2]:  1
Enter video file path [default: sample_video.mp4]:  datasets/le2i/Le2i_Sorted/Fall/Coffee_room_02/Videos/video (49).avi
Display video with pose estimation in real-time? (y/n) [default: y]:  y
Save output video? (y/n) [default: y]:  y



Running fall detection with:
- Weights: yolov7-w6-pose.pt
- Device: 0
- Source: datasets/le2i/Le2i_Sorted/Fall/Coffee_room_02/Videos/video (49).avi
- Display: Yes
- Save output: Yes



Confirm? (y/n) [default: y]:  y


Optimizer stripped from yolov7-w6-pose.pt, 161.1MB
Initializing Fall Detector with weights: yolov7-w6-pose.pt on device: 0
Fusing layers... 
Output will be saved to: output\video (49)_fall_detection.mp4
Starting fall detection...
Processing frame 1
Processing frame 2
Processing frame 3
Processing frame 4
Processing frame 5
Processing frame 6
Processing frame 7
Processing frame 8
Processing frame 9
Processing frame 10
Processing frame 11
Processing frame 12
Processing frame 13
Processing frame 14
Processing frame 15
Processing frame 16
Processing frame 17
Processing frame 18
Processing frame 19
Processing frame 20
Processing frame 21
Processing frame 22
Processing frame 23
Processing frame 24
Processing frame 25
Processing frame 26
Processing frame 27
Processing frame 28
Processing frame 29
Processing frame 30
Processing frame 31
Processing frame 32
Processing frame 33
Processing frame 34
Processing frame 35
Processing frame 36
Processing frame 37
Processing frame 38
Processing frame 39